## Setting up the environment

In [2]:
# authenticate EE
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=LFmjZpm-MY1BjREj3Z7AGahv8pZ6QHi0xPI7EBk8HAQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5c57jpTusFDEP3NiHWU8w1YKcj2Vpv4HmYvOCtWdC-pc6PKR87Yx8

Successfully saved authorization token.


In [3]:
# authenticate and mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import the Earth Engine API and initialize it.
import ee
from ee import batch
ee.Initialize()

## Functions

In [42]:
''' Check that Earth Engine is available '''
try:     # try to load the Landsat 8 Tier 1 EE image collection
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
except:  # if the loading image collection fails, then initialize EE in the code
    ee.Initialize()

'''
Functions to mask clouds based on pixel_qa band of Landsat 8 SR data borrowed from:
https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api

'''
# function to get QA data from image
def getQABits(image, start, end, mascara):
    # compute the bits we need to extract
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # return single band image of the extracted QA bits, giving the band a new name
    return image.select([0],[mascara]).bitwiseAnd(pattern).rightShift(start)

def maskQuality(image):
    # select the QA band
    QA = image.select('pixel_qa')
    # get the internal_cloud_algorithm_flag bit
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    # detect cirrus clouds using bit 9
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    # return an image masking out cloudy areas
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))


## Spatial and Temporal Filters and Other User Options

In [55]:
# create area of interest polygon (several options presented below)
## Example 1: Simple bounding box for the entire Gulf of Mexico
# aoipoly = ee.Geometry.BBox(-98, 26,-82,31)

## Example 2: Bounding polygon for area around Pensacola, FL, USA
aoipoly = ee.Geometry.Polygon([[[-88.412770, 31.098514], 
                                [-87.015281, 31.098514], 
                                [-87.015281, 30.156911], 
                                [-88.412770, 30.156911], 
                                [-88.412770, 31.098514]]])

## Example 3: Bounding polygon for entire Gulf of Mexico (USA) coast
# aoipoly = ee.Geometry.Polygon([[[-98.347274, 26.322097], 
#                                 [-98.238364, 28.187734], 
#                                 [-95.430534, 32.017493], 
#                                 [-84.011970, 32.564172], 
#                                 [-82.679402, 31.166281], 
#                                 [-80.410998, 24.154884], 
#                                 [-86.730146, 28.166929], 
#                                 [-94.552293, 27.623251], 
#                                 [-96.338964, 25.553449], 
#                                 [-98.347274, 26.322097]]])

# start and end dates for filtering imagery
START_DATE = '2016-01-01'
END_DATE = '2021-04-30'

# optional flag to enable data downloading
download_data = True

## Sample Data (download optional)

### Landsat 8 Tier 1 imagery

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR

In [59]:
# specify the image collection (Landsat 8), including its space and time filters
l8t1_tmp = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
                .filterDate(START_DATE, END_DATE)\
                .filterBounds(aoipoly)\
                .map(maskQuality)
# combine multiple images by taking the median of all available images, limit data
# to area of interest, and force 8-bit conversion
l8t1 = l8t1_tmp.median().clip(aoipoly)

if download_data:
  # create download container to bulk/batch export the download imagery to Drive
  l8t1_download = ee.batch.Export.image.toDrive(
      image = l8t1,              # specify data to be exported
      description = 'Landsat8_Tier1_imagery', # give the image a description
      folder = 'delta_dat',         # linked Google Drive folder
      region = aoipoly,             # limit output to specified geometry
      scale = 30,                   # spatial resolution of data
      fileNamePrefix = 'landsat8',  # filename for output(s)
      crs = 'EPSG:4326',            # specify projection
      maxPixels = 1e13,             # increase the number of pixel limit
      fileFormat = 'GeoTIFF',       # specify output file format
  )

  # start data download
  l8t1_download.start()

In [60]:
# check the status of the data download (if download_data is set to True)
l8t1_download.status()

{'creation_timestamp_ms': 1621614198793,
 'description': 'Landsat8_Tier1_imagery',
 'id': '5E4EJYFTMIA2N7UPHVQGSQEQ',
 'name': 'projects/earthengine-legacy/operations/5E4EJYFTMIA2N7UPHVQGSQEQ',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621614198793}

### US National Landcover

https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2016_REL

In [51]:
# EE location of NLCD land cover
nlcd = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')\
              .filterBounds(aoipoly)

# user-specified year of interest
yoi = '2016'

# filter data to 2016 and landcover band
nlcd2016 = nlcd.filter(ee.Filter.eq('system:index',yoi)).first()
landcover = nlcd2016.select('landcover').clip(aoipoly)

if download_data:
  # create download container to bulk/batch export the download imagery to Drive
  nlcd_download = ee.batch.Export.image.toDrive(
      image = landcover,
      description = 'Landcover',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'lulc',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  nlcd_download.start()

In [39]:
# check the status of the data download (if download_data is set to True)
nlcd_download.status()

{'attempt': 1,
 'creation_timestamp_ms': 1621611938641,
 'description': 'Landcover',
 'destination_uris': ['https://drive.google.com/#folders/1sbD68ZoklrtOlatavNR9RmJZG7MT4ovH'],
 'id': '3LVIC3IOSOG5YCAWHPHK2B5D',
 'name': 'projects/earthengine-legacy/operations/3LVIC3IOSOG5YCAWHPHK2B5D',
 'start_timestamp_ms': 1621611973041,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621612272619}

### MERIT DEM

Multi-Error-Removed Improved-Terrain (MERIT) DEM

https://developers.google.com/earth-engine/datasets/catalog/MERIT_DEM_v1_0_3

In [22]:
# EE location of MERIT DEM
mdem = ee.Image("MERIT/DEM/v1_0_3").clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  mdem_download = ee.batch.Export.image.toDrive(
      image = mdem,
      description = 'MERIT',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'merit_dem',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  mdem_download.start()

In [38]:
# check the status of the data download (if download_data is set to True)
mdem_download.status()

{'attempt': 1,
 'creation_timestamp_ms': 1621611945856,
 'description': 'MERIT',
 'destination_uris': ['https://drive.google.com/#folders/1sbD68ZoklrtOlatavNR9RmJZG7MT4ovH'],
 'id': 'I7V6QWZK3LP3AX43RJDYQJWI',
 'name': 'projects/earthengine-legacy/operations/I7V6QWZK3LP3AX43RJDYQJWI',
 'start_timestamp_ms': 1621612299344,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621612603966}

### Lithology

https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_US_lithology

In [56]:
# get lithology data
lith = ee.Image("CSP/ERGo/1_0/US/lithology")\
              .select('b1')\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  lith_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'lithology',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'lithology',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  lith_download.start()

In [58]:
# check the status of the data download (if download_data is set to True)
lith_download.status()

{'creation_timestamp_ms': 1621614179490,
 'description': 'lithology',
 'id': '3QSL3JR4ZYO5NMNJF4QBVBQG',
 'name': 'projects/earthengine-legacy/operations/3QSL3JR4ZYO5NMNJF4QBVBQG',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621614179490}

### NED Topographic Diversity

https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_US_topoDiversity

In [61]:
# get lithology data
tdi = ee.Image("CSP/ERGo/1_0/US/lithology")\
              .select('constant')\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  tdi_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'NED Topographic Diversity Index',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'tdi',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  tdi_download.start()

In [64]:
# check the status of the data download (if download_data is set to True)
tdi_download.status()

{'attempt': 1,
 'creation_timestamp_ms': 1621614306815,
 'description': 'NED Topographic Diversity Index',
 'id': 'BUS2JJD4K52WCUKGYU4UG6ND',
 'name': 'projects/earthengine-legacy/operations/BUS2JJD4K52WCUKGYU4UG6ND',
 'start_timestamp_ms': 1621614567472,
 'state': 'RUNNING',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621614570231}

## Cloud Analysis

In [ ]:
# landsat 8
l8t1
# landcover
landcover
# MERIT dem
mdem
# lithology
lith
# NED topographic diversity index
tdi